In [165]:
data = "../tabular_benchmark/data/bank/data.csv"

In [182]:
datas = "../tabular_benchmark/data/"

import os 

os.listdir("../tabular_benchmark/data/")

['bank', '.ipynb_checkpoints']

In [166]:
import os

os.listdir("../")

['tabular_benchmark', 'notebook', '.git']

In [153]:
import pandas as pd 




df = pd.read_csv(data, sep = ';')

object_columns_feat = cat_features = df.select_dtypes(include='object').columns.tolist()
"""

for c in object_columns_feat:
    df[c] = df[c].astype('int64')
"""
#object_columns_feat.remove("y")

"\n\nfor c in object_columns_feat:\n    df[c] = df[c].astype('int64')\n"

In [154]:


def convert_cat_str_to_int(df,cat_features):
    rule_dict = dict()
    for cat_feat in cat_features:
        uniques_ordered = sorted(df[cat_feat].unique().tolist())
        rule_dict[cat_feat] = rule = dict(zip(uniques_ordered,range(len(uniques_ordered))))
        df[cat_feat] = df[cat_feat].map(rule)
    return df, rule_dict




df, rule_dict = convert_cat_str_to_int(df,cat_features)


object_columns_feat.remove("y")

In [167]:
params = {"data_name":"bank", "link": "https://archive.ics.uci.edu/ml/datasets/Bank+Marketing#", "problem_type":"classification", "numerical_columns": ["age","balance","day", "duration", "pdays", "previous", "y"], "output":"y"}


import json

with open('../tabular_benchmark/data/bank/params.json', 'w') as fp:
    json.dump(params, fp)

    



In [176]:
import numpy as np
import optuna

import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).

data, target = df.loc[:, df.columns != 'y'],  df.loc[:, df.columns == 'y']
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)

dtrain = lgb.Dataset(train_x, label=train_y,   categorical_feature=object_columns_feat)
dvalid = lgb.Dataset(valid_x, label=valid_y,   categorical_feature=object_columns_feat)

    


def objective(trial):

    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback], categorical_feature=object_columns_feat)
    trial.set_user_attr(key="params", value=param)

    preds = gbm.predict(valid_x)
    
    

    
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy


study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="maximize"
)
study.optimize(objective, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-01-09 00:20:32,783] A new study created in memory with name: no-name-096b6195-7076-4a34-b714-8d705f787a4e
/Users/mehmetburaksayici/Desktop/venv/venv/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/mehmetburaksayici/Desktop/venv/venv/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
[I 2023-01-09 00:20:33,064] Trial 0 finished with value: 0.9071927806776962 and parameters: {'lambda_l1': 0.0009619768032890532, 'lambda_l2': 0.00027504982416930184, 'num_leaves': 19, 'feature_fraction': 0.7172286430235002, 'bagging_fraction': 0.8305307051749027, 'bagging_freq': 2, 'min_child_samples': 88}. Best is trial 0 with value: 0.9071927806776962.
/Users/mehmetburaksayici/Desktop/venv/venv/lib/python3.8/site-packages/lightgbm/basic.p

/Users/mehmetburaksayici/Desktop/venv/venv/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/mehmetburaksayici/Desktop/venv/venv/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
[I 2023-01-09 00:20:38,360] Trial 18 pruned. Trial was pruned at iteration 10.
[I 2023-01-09 00:20:38,443] Trial 19 pruned. Trial was pruned at iteration 10.


Number of finished trials: 20
Best trial:
  Value: 0.9109086083340706
  Params: 
    lambda_l1: 1.3243879726364816e-08
    lambda_l2: 5.602250971621139e-07
    num_leaves: 41
    feature_fraction: 0.9012434021993008
    bagging_fraction: 0.7208383132526704
    bagging_freq: 3
    min_child_samples: 69


In [177]:
best_model = study.best_trial.user_attrs

In [178]:
best_model

{'params': {'objective': 'binary',
  'metric': 'auc',
  'verbosity': -1,
  'boosting_type': 'gbdt',
  'lambda_l1': 1.3243879726364816e-08,
  'lambda_l2': 5.602250971621139e-07,
  'num_leaves': 41,
  'feature_fraction': 0.9012434021993008,
  'bagging_fraction': 0.7208383132526704,
  'bagging_freq': 3,
  'min_child_samples': 69}}

In [53]:
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback], categorical_feature=object_columns_feat)


NameError: name 'trial' is not defined

In [83]:
data, target = df.loc[:, df.columns != 'y'],  df.loc[:, df.columns == 'y']
target = target.astype("category")


train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
dtrain = lgb.Dataset(train_x, label=train_y,   categorical_feature="auto")
dvalid = lgb.Dataset(valid_x, label=valid_y,   categorical_feature="auto")


param = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "lambda_l1":   1e-8 ,
    "lambda_l2":   1e-8 ,
    "num_leaves":   256,
    "feature_fraction":  0.4 ,
    "bagging_fraction":   1.0,
    "bagging_freq":   7,
    "min_child_samples":   100
}

# Add a callback for pruning.
gbm = lgb.train(param, dtrain, valid_sets=[dvalid] , categorical_feature="auto" )

preds = gbm.predict(valid_x)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)

ValueError: DataFrame.dtypes for label must be int, float or bool

In [132]:
lgb.train(param, dtrain, valid_sets=[dvalid] , categorical_feature=object_columns_feat )

/Users/mehmetburaksayici/Desktop/venv/venv/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['contact', 'default', 'education', 'housing', 'job', 'loan', 'marital', 'month', 'poutcome', 'y']
  _log_warning('categorical_feature in Dataset is overridden.\n'


ValueError: DataFrame.dtypes for label must be int, float or bool

{'data_name': 'bank',
 'link': 'https://archive.ics.uci.edu/ml/datasets/Bank+Marketing#',
 'problem_type': 'classification',
 'numerical_columns': ['age',
  'balance',
  'day',
  'duration',
  'pdays',
  'previous'],
 'output': 'y'}

In [129]:
df, rule_dict = convert_cat_str_to_int(df,cat_features)

In [130]:
rule_dict

{'job': {'admin.': 0,
  'blue-collar': 1,
  'entrepreneur': 2,
  'housemaid': 3,
  'management': 4,
  'retired': 5,
  'self-employed': 6,
  'services': 7,
  'student': 8,
  'technician': 9,
  'unemployed': 10,
  'unknown': 11},
 'marital': {'divorced': 0, 'married': 1, 'single': 2},
 'education': {'primary': 0, 'secondary': 1, 'tertiary': 2, 'unknown': 3},
 'default': {'no': 0, 'yes': 1},
 'housing': {'no': 0, 'yes': 1},
 'loan': {'no': 0, 'yes': 1},
 'contact': {'cellular': 0, 'telephone': 1, 'unknown': 2},
 'month': {'apr': 0,
  'aug': 1,
  'dec': 2,
  'feb': 3,
  'jan': 4,
  'jul': 5,
  'jun': 6,
  'mar': 7,
  'may': 8,
  'nov': 9,
  'oct': 10,
  'sep': 11},
 'poutcome': {'failure': 0, 'other': 1, 'success': 2, 'unknown': 3},
 'y': {'no': 0, 'yes': 1}}

In [131]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,0,17,9,977,3,-1,0,3,1
45207,71,5,0,0,0,1729,0,0,0,17,9,456,2,-1,0,3,1
45208,72,5,1,1,0,5715,0,0,0,17,9,1127,5,184,3,2,1
45209,57,1,1,1,0,668,0,0,1,17,9,508,4,-1,0,3,0


In [159]:

import xgboost as xgb


def objective(trial):

    data, target = df.loc[:, df.columns != 'y'],  df.loc[:, df.columns == 'y']

    
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y, enable_categorical = True)
    dvalid = xgb.DMatrix(valid_x, label=valid_y , enable_categorical = True)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy



In [160]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-01-08 23:47:38,421] A new study created in memory with name: no-name-3d0c66a9-e565-4b94-ad3d-671825448586
[I 2023-01-08 23:47:38,570] Trial 0 finished with value: 0.8927718304874812 and parameters: {'booster': 'gbtree', 'lambda': 0.0015363259645493103, 'alpha': 1.2517118592479201e-06, 'subsample': 0.9164619919312549, 'colsample_bytree': 0.6455034000829812, 'max_depth': 7, 'min_child_weight': 8, 'eta': 1.4704458005381896e-05, 'gamma': 6.66808018472826e-08, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.8927718304874812.
[I 2023-01-08 23:47:38,662] Trial 1 finished with value: 0.892240997965142 and parameters: {'booster': 'gbtree', 'lambda': 1.209064178019936e-06, 'alpha': 0.011327292870551847, 'subsample': 0.9096288941918724, 'colsample_bytree': 0.6526737529597253, 'max_depth': 5, 'min_child_weight': 7, 'eta': 1.4510996485615787e-05, 'gamma': 5.772720734719733e-05, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.8927718304874812.
[I 2023-01-08 23:47:38,736

[I 2023-01-08 23:47:40,874] Trial 17 finished with value: 0.905157922675396 and parameters: {'booster': 'dart', 'lambda': 1.3866243326113807e-08, 'alpha': 8.254790973701976e-05, 'subsample': 0.6425195491797724, 'colsample_bytree': 0.9298333654262717, 'max_depth': 7, 'min_child_weight': 2, 'eta': 0.008552244837682528, 'gamma': 0.04037301612100824, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.00032560541720132435, 'skip_drop': 6.189094820129333e-06}. Best is trial 17 with value: 0.905157922675396.
[I 2023-01-08 23:47:41,031] Trial 18 finished with value: 0.884455454304167 and parameters: {'booster': 'dart', 'lambda': 9.855102379286908e-05, 'alpha': 2.953654313674472e-05, 'subsample': 0.6394556646215463, 'colsample_bytree': 0.4014295375348377, 'max_depth': 7, 'min_child_weight': 3, 'eta': 0.008245710799192582, 'gamma': 0.04344419238426504, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.001536

[I 2023-01-08 23:47:43,209] Trial 33 finished with value: 0.8914447491816332 and parameters: {'booster': 'dart', 'lambda': 2.3261586770513266e-06, 'alpha': 4.221320808131864e-05, 'subsample': 0.8776479667414449, 'colsample_bytree': 0.6153210813950564, 'max_depth': 5, 'min_child_weight': 8, 'eta': 0.023563811979608693, 'gamma': 1.0660761232325352e-08, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.0003551556950641734, 'skip_drop': 0.1518917209912698}. Best is trial 22 with value: 0.9052463947624525.
[I 2023-01-08 23:47:43,382] Trial 34 finished with value: 0.8995841811908343 and parameters: {'booster': 'dart', 'lambda': 3.2906805697703256e-05, 'alpha': 0.0011380489276457443, 'subsample': 0.9984941464309112, 'colsample_bytree': 0.8464079709410333, 'max_depth': 7, 'min_child_weight': 6, 'eta': 0.17481383666927772, 'gamma': 0.0004050015753399802, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 2.57

[I 2023-01-08 23:47:45,124] Trial 51 finished with value: 0.8864903123064674 and parameters: {'booster': 'gblinear', 'lambda': 4.57813695480354e-08, 'alpha': 0.00020381624210990764, 'subsample': 0.5378760469828127, 'colsample_bytree': 0.6979515618999436}. Best is trial 41 with value: 0.9067504202424135.
[I 2023-01-08 23:47:45,289] Trial 52 finished with value: 0.8958683535344599 and parameters: {'booster': 'dart', 'lambda': 4.677242677639975e-05, 'alpha': 6.541472082053516e-05, 'subsample': 0.6168299866319654, 'colsample_bytree': 0.6290755792538787, 'max_depth': 7, 'min_child_weight': 5, 'eta': 0.000932080386032823, 'gamma': 0.000669491733273035, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 9.189404982745425e-08, 'skip_drop': 4.427533553793962e-05}. Best is trial 41 with value: 0.9067504202424135.
[I 2023-01-08 23:47:45,450] Trial 53 finished with value: 0.8958683535344599 and parameters: {'booster': 'dart', 'lambda': 1.9674485352949862e-

[I 2023-01-08 23:47:47,440] Trial 67 finished with value: 0.9005573741484562 and parameters: {'booster': 'dart', 'lambda': 5.718467672003534e-06, 'alpha': 0.00022030103079613786, 'subsample': 0.8208742314739395, 'colsample_bytree': 0.8972166030441958, 'max_depth': 7, 'min_child_weight': 6, 'eta': 0.0005030214930312119, 'gamma': 0.00013394857350073616, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.855271822209309, 'skip_drop': 0.00017038769603424715}. Best is trial 41 with value: 0.9067504202424135.
[I 2023-01-08 23:47:47,570] Trial 68 finished with value: 0.8945412722286119 and parameters: {'booster': 'dart', 'lambda': 9.620130666063178e-08, 'alpha': 0.007900685787385007, 'subsample': 0.9521320517126666, 'colsample_bytree': 0.835625741146057, 'max_depth': 5, 'min_child_weight': 6, 'eta': 0.020716950205758384, 'gamma': 0.02237860510523659, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.04

[I 2023-01-08 23:47:50,193] Trial 83 finished with value: 0.9041847297177741 and parameters: {'booster': 'dart', 'lambda': 1.0587794116169837e-08, 'alpha': 0.1440380352175111, 'subsample': 0.5785893179772459, 'colsample_bytree': 0.9210582622947703, 'max_depth': 7, 'min_child_weight': 9, 'eta': 2.8588976965444863e-06, 'gamma': 0.9073603272281561, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.5151159177702381, 'skip_drop': 2.1474554025619298e-08}. Best is trial 74 with value: 0.9090506945058834.
[I 2023-01-08 23:47:50,378] Trial 84 finished with value: 0.8968415464920817 and parameters: {'booster': 'dart', 'lambda': 0.0005918435385257874, 'alpha': 0.07097247189346505, 'subsample': 0.7219357545028786, 'colsample_bytree': 0.8587492426166369, 'max_depth': 7, 'min_child_weight': 9, 'eta': 2.248253467131345e-07, 'gamma': 1.1456099158763911e-05, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.

Number of finished trials:  100
Best trial:
  Value: 0.9090506945058834
  Params: 
    booster: dart
    lambda: 1.1610032354870255e-06
    alpha: 0.06642648138648113
    subsample: 0.6872012415625147
    colsample_bytree: 0.9649865823227946
    max_depth: 7
    min_child_weight: 10
    eta: 0.0022355174557013645
    gamma: 3.94017568583284e-05
    grow_policy: lossguide
    sample_type: weighted
    normalize_type: forest
    rate_drop: 0.2618260900859333
    skip_drop: 1.4662474936815081e-08


In [63]:
df.loc[:, df.columns == 'y']

,y
0,no
1,no
2,no
3,no
4,no
...,...
45206,yes
45207,yes
45208,yes
45209,no


In [68]:
data, target = df.loc[:, df.columns != 'y'],  df.loc[:, df.columns == 'y']
target = target.astype("category")


train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
dtrain = xgb.DMatrix(train_x, label=train_y, enable_categorical = True)
dvalid = xgb.DMatrix(valid_x, label=valid_y , enable_categorical = True)

param = {
    "verbosity": 0,
    "objective": "binary:logistic",
    # use exact for small dataset.
    "tree_method": "exact",
    # defines booster, gblinear for linear functions.
    "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
    # L2 regularization weight.
    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    # L1 regularization weight.
    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    # sampling ratio for training data.
    "subsample": trial.suggest_float("subsample", 0.2, 1.0),
    # sampling according to each tree.
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
}

if param["booster"] in ["gbtree", "dart"]:
    # maximum depth of the tree, signifies complexity of the tree.
    param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
    # minimum child weight, larger the term more conservative the tree.
    param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
    param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
    # defines how selective algorithm is.
    param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
    param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

if param["booster"] == "dart":
    param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
    param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
    param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
    param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

bst = xgb.train(param, dtrain)
preds = bst.predict(dvalid)
pred_labels = np.rint(preds)





ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:y: category

In [74]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no
